[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/gaussian-splatting-colab/blob/main/gaussian_splatting_viewer_old_colab.ipynb)

In [ ]:
# https://docs.nvidia.com/video-technologies/video-codec-sdk/12.0/ffmpeg-with-nvidia-gpu/index.html#compiling-for-linux

%cd /content
!git clone https://git.videolan.org/git/ffmpeg/nv-codec-headers
%cd /content/nv-codec-headers
!make install
%cd /content
!git clone https://git.ffmpeg.org/ffmpeg
%cd /content/ffmpeg
!apt install build-essential yasm cmake libtool libc6 libc6-dev unzip wget libnuma1 libnuma-dev
!./configure --enable-nonfree --enable-cuda-nvcc --enable-libnpp --extra-cflags=-I/usr/local/cuda/include --extra-ldflags=-L/usr/local/cuda/lib64 --disable-static --enable-shared
!make -j 8
!make install
!ldconfig /usr/local/lib
!ffmpeg -version

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/gaussian-viewer
%cd /content/gaussian-viewer

!pip install https://github.com/camenduru/gaussian-splatting-colab/releases/download/colab/PyNvCodec-0.1.0-cp310-cp310-linux_x86_64.whl
!pip install https://github.com/camenduru/gaussian-splatting-colab/releases/download/colab/PytorchNvCodec-0.0.0-cp310-cp310-linux_x86_64.whl
!pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.whl
!pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/simple_knn-0.0.0-cp310-cp310-linux_x86_64.whl

!pip install -q aioice aiortc annotated-types av dnspython fastapi h11 httptools ifaddr 
!pip install -q nvidia-cublas-cu11 nvidia-cuda-cupti-cu11 nvidia-cuda-nvrtc-cu11 nvidia-cuda-runtime-cu11 
!pip install -q nvidia-cudnn-cu11 nvidia-cufft-cu11 nvidia-curand-cu11 nvidia-cusolver-cu11 nvidia-cusparse-cu11
!pip install -q nvidia-nccl-cu11 nvidia-nvtx-cu11 plyfile pydantic_core pyee pylibsrtp python-dotenv starlette 
!pip install -q twilio uvicorn uvloop watchfiles websockets

!mkdir -p /content/gaussian-viewer/models/bicycle/point_cloud/iteration_30000
!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/train/point_cloud/iteration_30000/point_cloud.ply -O /content/gaussian-viewer/models/bicycle/point_cloud/iteration_30000/point_cloud.ply
!wget https://huggingface.co/camenduru/gaussian-splatting/raw/main/train/cameras.json -O /content/gaussian-viewer/models/bicycle/cameras.json 

!git clone -b dev https://github.com/camenduru/gaussian-viewer-frontend /content/gaussian-viewer/gaussian-viewer-frontend

In [ ]:
%cd /content
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O /content/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8000, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)

In [ ]:
%cd /content/gaussian-viewer
!uvicorn main:app